In [36]:
import numpy as np

#settings for the model
isPoly=False
isLeanear=True
PolyDegree=2
usingRegularizer=True
Lambda=0.01
W=None # current weight
def my_regression(trainX,testX,noutputs):
    ## coding here
    phi=None
    if isLeanear:
        phi=trainX[:,0:trainX.shape[1]-noutputs]
        ones=np.ones((trainX.shape[0],1))
        phi=np.append(ones,phi,axis=1)
        print(phi)
        T=trainX[:,trainX.shape[1]-noutputs:]
        if usingRegularizer==True:
            W=np.dot(np.dot(np.linalg.pinv( Lambda * np.eye(phi.shape[1]) + np.dot(phi.T,phi)),phi.T),T)
        else:
            W=np.dot(np.dot(np.linalg.pinv(np.dot(phi.T,phi)),phi.T),T)
        print(T)
        print(W)
        outputs=np.dot(phi,W)
        print(outputs)
        print("leanear")
        return np.dot(np.append(np.ones((testX.shape[0],1)),testX,axis=1),W)
    elif isPoly:
        print("poly")
    

    

    pass


##main:

if __name__=="__main__":
    trainX=np.array([[1,2,4],[2,3,7],[3,4,10],[9,100,118]])
    testX=np.array([[1,1],[2,2],[3,3]])
    noutputs=1
    testOut=my_regression(trainX,testX,noutputs)
    print(testOut)

    pass


[[  1.   1.   2.]
 [  1.   2.   3.]
 [  1.   3.   4.]
 [  1.   9. 100.]]
[[  4]
 [  7]
 [ 10]
 [118]]
[[0.01939327]
 [1.98906544]
 [1.0007913 ]]
[[  4.0100413 ]
 [  6.99989803]
 [  9.98975477]
 [118.00011196]]
leanear
[[3.00925   ]
 [5.99910674]
 [8.98896347]]
